In [4]:
## import
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import requests
import json
import japanize_matplotlib

japanize_matplotlib.japanize()
import warnings
import graphviz
import lingam
from lingam.utils import make_dot, print_causal_directions, print_dagc
from statsmodels.formula.api import ols
from scipy.stats import t
from linearmodels.panel import PanelOLS, PooledOLS, compare, FirstDifferenceOLS
import itertools
from lmfit.models import PowerLawModel

import lmfit as lf
from sklearn.linear_model import LinearRegression
from linearmodels import IVSystemGMM
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from pydynpd import regression
from sklearn.preprocessing import StandardScaler
import math

warnings.resetwarnings()
warnings.simplefilter("ignore", FutureWarning)

In [2]:
# 統計情報の取得
prev = ['001','015','016','021','022','023','036','037','038','040','041','219','042','043','044','045','046','047','048','049','050','051','052','053','054','055','056','220','058','059','060','061','062','063','065','057','066','235','067','068','069','070','071','072','073','074','075','086','100','121','122','128','130','138','142','144','145','146','147','148','149','150','151','152','153','154','155','217','224','157','225','226','237','131','135','119','136','137','134','120','133','126','125','123','127','143','124','132','129' ]
prev_cat2="101,162,102,103,135,136,131,132,133,127,128,151,138,143,109,110,163,115,116,118,120,121,124,122,145,107,135,136,130,150,139,148,140,141,142,138,153,152"
prev_cat2_list = prev_cat2.split(',')
create_cat2 = "109,110,163,111,112,113,114,115,116,117,118,119,120,121,124,122,145,123,125,126,106,153,152"
create_cat2_list = create_cat2.split(',')
add_list =list(set(create_cat2_list)-set(prev_cat2_list))
new_list = prev_cat2_list + add_list

def getCSV():
    url = f"http://api.e-stat.go.jp/rest/3.0/app/getSimpleStatsData"

    #
    params = {
        "cdCat01": ",".join(prev),
        "cdTimeFrom": "20040",
        "cdTimeTo": "20190",
        "cdCat02": ",".join(new_list),
        "cdCat03": "25,24,21,18,16",
        "appId": "5dc8d291344294b140960da279a4cae759394794",
        "lang": "J",
        "statsDataId": "0003060791",
        "limit": "100000",
    }

    def download_csv(url, filepath="new.csv", enc="utf-8", dec="utf-8", logging=False):
        try:
            if logging:
                print("HTTP GET", url)
            next_key = 1
            while not np.isnan(float(next_key)):
                print(next_key)
                params["startPosition"] = next_key
                r = requests.get(url, params, stream=True)
                with open(filepath, "w") as f:
                    f.write(r.text)
                reader = csv.reader(r.text)
                df_r = pd.read_csv(
                    filepath,
                    names=[
                        "cat01_code",
                        "調査項目 (金融業、保険業以外の業種)",
                        "cat02_code",
                        "業種 (金融業、保険業以外の業種)",
                        "cat03_code",
                        "規模 (金融業、保険業以外の業種)",
                        "time_code",
                        "年  期",
                        "unit",
                        "value",
                        "annotation",
                    ],
                    encoding="utf-8",
                )
                if next_key == 1:
                    df = pd.DataFrame(df_r.iloc[28:, :10], columns=df_r.iloc[27, :10])
                else:
                    df = pd.concat([df, df_r.iloc[28:, :10]])
                if any(df_r.loc[8] == "NEXT_KEY"):
                    next_key = df_r.iloc[7, 1]
                else:
                    next_key = np.nan
            return df
        except requests.exceptions.RequestException as error:
            print(error)

    df_csv = download_csv(url)
    df_csv.to_csv("df.csv")

getCSV()

1


/var/folders/q2/cb1gg0616kj1gk7v7d7nh5bh0000gq/T/ipykernel_76973/236101311.py:38: DtypeWarning: Columns (0,2,4,6,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_r = pd.read_csv(


100000


/var/folders/q2/cb1gg0616kj1gk7v7d7nh5bh0000gq/T/ipykernel_76973/236101311.py:38: DtypeWarning: Columns (0,2,4,6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_r = pd.read_csv(


199999


/var/folders/q2/cb1gg0616kj1gk7v7d7nh5bh0000gq/T/ipykernel_76973/236101311.py:38: DtypeWarning: Columns (0,2,4,6,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_r = pd.read_csv(


In [5]:
def createDF():
    df_csv = pd.read_csv("df.csv", index_col=0).reset_index(drop=True)
    df = df_csv.iloc[0:0]
    for item in df_csv["調査項目 (金融業、保険業以外の業種)"].unique():
        if df.empty:
            df = pd.DataFrame(
                {
                    "year": df_csv[df_csv["調査項目 (金融業、保険業以外の業種)"] == item]["time_code"]
                    .reset_index(drop=True)
                    .astype(str)
                    .str[:-1]
                    .astype(int),
                    "industry_name": df_csv[df_csv["調査項目 (金融業、保険業以外の業種)"] == item][
                        "業種 (金融業、保険業以外の業種)"
                    ].reset_index(drop=True),
                    "industry": df_csv[df_csv["調査項目 (金融業、保険業以外の業種)"] == item][
                        "cat02_code"
                    ].reset_index(drop=True),
                    "scale": df_csv[df_csv["調査項目 (金融業、保険業以外の業種)"] == item][
                        "cat03_code"
                    ].reset_index(drop=True),
                }
            )
        elif df_csv[df_csv["調査項目 (金融業、保険業以外の業種)"] == item].iloc[0]["unit"] == "百万円":
            column = (
                df_csv[df_csv["調査項目 (金融業、保険業以外の業種)"] == item]["value"]
                .reset_index(drop=True)
                .replace("*", "0")
                .astype(int)
                * 10e2
            )
            df = pd.concat(
                [
                    df,
                    pd.Series(
                        column,
                        name=item,
                    ),
                ],
                axis=1,
            )
            if all(column > 0):
                df = pd.concat(
                    [
                        df,
                        pd.Series(
                            np.log(column),
                            name="l" + item,
                        ),
                    ],
                    axis=1,
                )
        elif df_csv[df_csv["調査項目 (金融業、保険業以外の業種)"] == item].iloc[0]["unit"] == "万円":
            column = (
                df_csv[df_csv["調査項目 (金融業、保険業以外の業種)"] == item]["value"]
                .reset_index(drop=True)
                .replace("*", "0")
                .astype(int)
            )
            df = pd.concat(
                [
                    df,
                    pd.Series(
                        column,
                        name=item,
                    ),
                ],
                axis=1,
            )
            if all(column > 0):
                df = pd.concat(
                    [
                        df,
                        pd.Series(
                            np.log(column),
                            name="l" + item,
                        ),
                    ],
                    axis=1,
                )
        else:
            df = pd.concat(
                [
                    df,
                    pd.Series(
                        df_csv[df_csv["調査項目 (金融業、保険業以外の業種)"] == item][
                            "value"
                        ].reset_index(drop=True),
                        name=item,
                    )
                    .replace("*", "0")
                    .replace("***", "")
                    .astype(float),
                ],
                axis=1,
            )
        df.replace("***", "", inplace=True)
        df.replace("**", "", inplace=True)
        df.replace("*", "", inplace=True)
        df.fillna(0, inplace=True)
    return df


def dropOutlier(df):
    for column in [
        "software",
        "value",
        "value_rate",
        "soft_rate",
    ]:
        des = df[column].describe()
        rng = des.loc["75%"] - des.loc["25%"]
        uplimit = des.loc["75%"] + 1.724 * rng
        lowlimit = des.loc["25%"] - 1.724 * rng
        if any(~((df[column] < uplimit) | (df[column] > lowlimit))):
            print(column)
            print(df[~((df[column] < uplimit) | (df[column] > lowlimit))])
        df = df[(df[column] < uplimit) | (df[column] > lowlimit)]
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(how="any", inplace=True)
    return df


## 処置の処理
def CreateTreat(df, split: str, soft="", year=2015):
    if soft == "rate":
        df["sft_diff"] = df.groupby(["scale", "industry"])["soft_rate"].diff()
    elif soft == "log":
        df["sft_diff"] = df.groupby(["scale", "industry"])["log_software"].diff()
    else:
        df["sft_diff"] = df.groupby(["scale", "industry"])["software"].diff()
    df.loc[(df.year > year + 3) | (df.year < year), "sft_diff"] = np.nan  # type: ignore
    if split == "mean":
        split = df["sft_diff"].mean()
    elif split == "75":
        split = df["sft_diff"].describe()["75%"]
    else:
        split = df["sft_diff"].median()
    print(split)
    # print(df["sft_diff"].describe())
    df["group"] = 0
    df.sort_values(by="year", inplace=True, ascending=False)
    for index, row in df[df["sft_diff"] < split].iterrows():
        df.loc[
            (df["industry"] == row.industry) & (df["scale"] == row.scale), "treat"
        ] = 0
    for index, row in df[df["sft_diff"] >= split].iterrows():
        df.loc[
            (df["industry"] == row.industry) & (df["scale"] == row.scale), "treat"
        ] = 1
        df.loc[
            (df["industry"] == row.industry) & (df["scale"] == row.scale), "group"
        ] = row.year
    df.sort_values(by="year", inplace=True)
    return df


def getDummey(df):
    df_year = df["year"].copy()
    df = pd.get_dummies(df, drop_first=True, columns=["year"])
    df["year"] = df_year
    return df

# CreateTreat(df,'mean')

# データの前処理関数
def dataCleanig(out):
    # csvからデータを取得
    df = createDF()
    df.rename(columns=lambda s: s.replace("(当期末)", ""), inplace=True)
    df.rename(
        columns={
            "ソフトウェア(当期末固定資産)": "software",
            "付加価値": "value",
            "資本金": "capital",
            "無形固定資産": "infasset",
            "固定資産": "fasset",
            "有形固定資産": "tfasset",
            # "ソフトウェアを除く設備投資(当期末資金需給)": "woscinvest",
            "ソフトウェアを除く無形固定資産(当期末固定資産)": "wosinfasset",
            "固定比率": "frate",
            "労働装備率": "labar_asset_rate",
            "自己資本比率": "self_asset_rate",
            "総資本回転率": "tac_rate",
            "配当性向": "payout_ratio",
            "資本剰余金": "surplus",
            "社内留保": "retain",
            "減価償却費": "depreciation",
            "特別減価償却費": "special_depreciation",
            "従業員給与": "income",
            "設備投資効率": "invest_effect_rate",
        },
        inplace=True,
    )
    df.replace("繊維工業(H20年度まで)", "繊維工業", inplace=True)
    df.replace("衣服・その他の繊維製品製造業(H20年度まで)", "繊維工業", inplace=True)
    ag_industry_num = df[df["industry_name"] == "農業、林業"].industry.unique()
    df.replace("林業(H20年度まで)", "農業、林業", inplace=True)
    df.replace("農業(H20年度まで)", "農業、林業", inplace=True)
    df["industry"] = (
        df["industry"]
        .replace(110, 163)
        .replace(111, 163)
        .replace(102, ag_industry_num[0])
        .replace(162, ag_industry_num[0])
    )
    df = df.groupby(["scale", "year", "industry_name", "industry"]).mean().reset_index()
    # 規模を大きさ比較できるように
    df["scale"] = df["scale"].map({16: 5, 21: 75, 24: 5 * 10**2, 25: 10**3, 18: 25})
    df["old_industry_name"] = df["industry_name"]
    df["old_industry"] = df["industry"]
    df.loc[
        df[
            (
                (df["industry"] < 123)
                & (~df["industry"].astype(str).str.contains("103|101|106|105|106|107"))
            )
            | (df["industry"] == 163)
            | (df["industry_name"].str.contains("製造業"))
        ].index,
        "industry_name",
    ] = "製造業"
    df.loc[
        df[
            (df["industry_name"] == "電気業") | (df["industry_name"] == "ガス・熱供給・水道業")
        ].index,
        "industry_name",
    ] = "インフラ業"
    df.loc[
        df[
            (df["industry_name"] == "農業、林業")
            | (df["industry_name"] == "漁業")
        ].index,
        "industry_name",
    ] = "農林水産業"
    df.loc[
        df[
            (df["industry_name"] == "陸運業")
            | (df["industry_name"] == "水運業")
            | (df["industry_name"] == "その他の運輸業")
        ].index,
        "industry_name",
    ] = "運輸業 郵便業"
    df.loc[
        df[(df["industry_name"] == "卸売業") | (df["industry_name"] == "小売業")].index,
        "industry_name",
    ] = "卸売業 小売業"
    df.loc[
        df[
            (df["industry_name"] == "不動産業")
            | (df["industry_name"] == "リース業")
            | (df["industry_name"] == "その他の物品賃貸業")
        ].index,
        "industry_name",
    ] = "不動産業 物品賃貸業"
    df.loc[
        df[(df["industry_name"] == "生活関連サービス業") | (df["industry_name"] == "娯楽業")].index,
        "industry_name",
    ] = "生活関連サービス 娯楽業"
    df.loc[
        df[(df["industry_name"] == "宿泊業") | (df["industry_name"] == "飲食サービス業")].index,
        "industry_name",
    ] = "宿泊 飲食サービス業"
    df.loc[
        df[(df["industry_name"] == "広告業") | (df["industry_name"] == "その他のサービス業")].index,
        "industry_name",
    ] = "その他のサービス業"
    df["industry_name"] = df["industry_name"].str.replace('_',' ')
    # 内部留保率の欠損値の置換
    df["retain_rate"] = df["retain"].astype(int) / df["当期純利益"].astype(int)
    df["avg_income"] = df["income"].astype(int) / df["期中平均従業員数"].astype(int)
    # 減価償却費
    df["depreciation"] = df["depreciation"] + df["special_depreciation"]
    # 　ソフトウェア除く設備投資
    df["woscinvest"] = df["tfasset"] + df["wosinfasset"] - df["depreciation"]
    # 固定比率
    df["frate"] = df["fasset"] / df["自己資金(当期末資金需給)"]
    # 無形固定資産に占める割合を計算（普通の固定資産だと割合が低くなりすぎて比較しずらいため無形固定資産にした）
    df["soft_rate"] = df["software"].astype(int) / df["売上高"].astype(int)
    df["invest_rate"] = df["woscinvest"].astype(int) / df["売上高"].astype(int)
    df["income_rate"] = df["income"].astype(int) / df["売上高"].astype(int)
    # 流動比率
    df["flow_rate"] = df["流動資産"].astype(int) / df["流動負債"].astype(int)
    # 付加価値率再計算
    df["value_rate"] = df["value"].astype(int) / df["売上高"].astype(int)
    df["tfasset_rate"] = df["tfasset"].astype(int) / df["fasset"].astype(int)
    # df['income_diff']= df['income']
    # 対数の列を追加
    shift_list = [
        "software",
        "avg_income",
        "depreciation",
        "soft_rate",
        "tfasset_rate",
        "capital",
        "surplus",
        "tfasset",
        "wosinfasset",
        "retain",
        "income_diff",
        "woscinvest",
        "invest_rate",
        "retain_rate",
        "labar_asset_rate",
        "income_rate",
        "software_diff",
        "woscinvest_diff",
        "income"
    ]
    diff = ["software", "woscinvest", "income"]
    df_diff = df.groupby(["industry", "scale"])[diff].diff()
    df["value_diff"] = df.groupby(["industry", "scale"])["value"].diff()
    # マージ
    df = pd.merge(
        df, df_diff, left_index=True, right_index=True, suffixes=("", "_diff")
    )
    for column in shift_list:
        df[f"{column}0"] = df.groupby(["industry", "scale"])[column].shift(-1)
        df[f"{column}1"] = df.groupby(["industry", "scale"])[column].shift()
        df[f"{column}2"] = df.groupby(["industry", "scale"])[column].shift(2)
        # df[f"{column}3"] = df.groupby(["industry", "scale"])[column].shift(3)
        if ~column.endswith("_rate|_diff"):
            df[f"l{column}"] = np.log(df[f"{column}"])
            df[f"l{column}1"] = np.log(df[f"{column}1"])
            df[f"l{column}2"] = np.log(df[f"{column}2"])
            # df[f"l{column}3"] = np.log(df[f"{column}3"])
    df = df.replace([np.inf, -np.inf], np.nan)
    df.fillna(0, inplace=True)
    # 従業員数の役員数で置換
    df.loc[df[df["期中平均従業員数"].astype(int) == 0].index, "期中平均従業員数"] = df[
        df["期中平均従業員数"].astype(int) == 0
    ]["期中平均役員数"].astype(int)
    # 欠損値の検出
    for column in df.columns:
        if any(df[column] == np.inf):
            print(df[df[column] == np.inf][column])
        if any(df[column] == np.nan):
            print(df[df[column] == np.nan][column])
        if any(df[column].isnull()):
            print(df[df[column].isnull()][column])
        if any(df[column].isna()):
            print(df[df[column].isna()][column])
    if out == True:
        df = dropOutlier(df)
    # df.industry_name = df.industry_name.str.replace("、|・", "_", regex=True)
    df.industry_name = df.industry_name.str.replace(r"\(集約\)", "")
    df["category_name"] = "a"
    for scale, industry in itertools.product(
        df["scale"].unique(), df["industry_name"].unique()
    ):
        df.loc[
            (df["scale"] == scale) & (df["industry_name"] == industry), "category_name"
        ] = (industry + "_" + str(scale))

    df["id"] = (df["old_industry"].astype(int) * df["scale"].astype(int)).astype(str)
    year = df.year
    df = pd.get_dummies(df, drop_first=True, columns=["year"])
    df["year"] = year
    df["category"] = df["category_name"].astype("category").cat.codes
    scale = df.scale
    industry_name = df.industry_name.astype("category")
    df.scale = df.scale.astype(int)
    df = pd.get_dummies(df, columns=["industry_name", "scale"])
    df["scale"] = scale
    df["industry_name"] = industry_name
    for column in df.columns[df.columns.str.startswith(("industry_name_", "scale_"))]:
        df["soft_" + column] = df[column] * df.software
        df["1" + "soft_" + column] = df[column] * df.software1
        df["2" + "soft_" + column] = df[column] * df.software2
        # df["3" + "soft_" + column] = df[column] * df.software3
    # sort
    df.sort_values(["industry", "scale", "year"], inplace=True)
    # indexをリセット
    df = df.reset_index(drop=True)
    return df


df = dataCleanig(out=False)
# 　標準化
# 標準化のクラスを定義
ms = StandardScaler()
# fit_transformを用いて、特徴量の平均値と標準化を計算してから特徴量のスケール変換を行う
df_std = pd.DataFrame(
    ms.fit_transform(
        df.drop(
            columns=[
                "industry_name",
                "category",
                "category_name",
                "old_industry",
                "old_industry_name",
                "id",
            ]
        )
    ),
    index=df.index,
    columns=df.drop(
        columns=[
            "industry_name",
            "category",
            "category_name",
            "old_industry",
            "old_industry_name",
            "id",
        ]
    ).columns,
)

/var/folders/q2/cb1gg0616kj1gk7v7d7nh5bh0000gq/T/ipykernel_83551/3563677249.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv("df.csv", index_col=0).reset_index(drop=True)
/Users/ayuhi2/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ayuhi2/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ayuhi2/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ayuhi2/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **

In [194]:
# 多重共線性の確認　１０以下だったらOK
df_vif = df.loc[
    :, ["software", "woscinvest", "income"]
]
df_vif["Intercept"] = 1.0

for i in range(df_vif.columns.size - 1):  # 定数項は無視するために-1
    name = df_vif.columns[i]
    vif_val = vif(df_vif.to_numpy(), i)
    print(f"{name : <10} {vif_val}")

software   1.1285623454752873
woscinvest 1.2687510865673137
income     1.2913145337473004


In [3]:
def getformula(num=""):
    industry_name_list = df.columns[df.columns.str.startswith(num + "soft_i")].values
    industry_formula = " + ".join(industry_name_list)
    scale_name_list = df.columns[df.columns.str.startswith(num + "soft_s")].values

    scale_formula = " + ".join(scale_name_list)
    return industry_formula, scale_formula

def getdf():
    df_did = df.copy()
    return df_did

def getYearFormula(ago = 0):
    year_list = df.columns[df.columns.str.startswith("year_")].tolist()
    if ago < 0:
        year = 2019 + 1 + ago
        for i in np.arange(year,2019 + 1):
            year_list.remove("year_" + str(year))
    else:
        year = 2004 + ago
        for i in range(year - 2004):
            year_list.remove("year_" + str(int(year) - i))
    year_formula = " + ".join(year_list)
    return year_formula
def getScaleDummyformula():
    scale_name_list = df.columns[df.columns.str.startswith("scale_")].values
    scale_formula = " + ".join(scale_name_list)
    return scale_formula

def getDummyformula(num=""):
    industry_name_list = df.columns[
        df.columns.str.startswith(num + "industry_name_")
    ].values
    industry_formula = " + ".join(industry_name_list)
    scale_name_list = df.columns[df.columns.str.startswith(num + "scale_")].values
    scale_formula = " + ".join(scale_name_list)
    return industry_formula, scale_formula


In [20]:
import graphviz
ago = 1
df_fd = getdf()
if ago == -1:
    df_fd = df_fd[df_fd.year <= 2019 + ago]
else:
    df_fd = df_fd[df_fd.year >= 2004 + ago]
df_fd = df_fd.set_index(["id", "year"])
str_ago = "" if ago == 0 else "0" if ago == -1 else ago
year_formula = getYearFormula(ago)
formula_fe = f"value ~ woscinvest{str_ago} + software{str_ago} + income{str_ago} + EntityEffects + TimeEffects"
result_fe = PanelOLS.from_formula(formula_fe, data=df_fd).fit(
    cov_type="clustered", cluster_entity=True
)
f = graphviz.Graph('model2', filename='path.gv') 
params = result_fe.params.iloc[:3].rename(index={"income1": "一年前の従業員給与","software1":"一年前のソフトウェア","woscinvest1": "一年前の設備投資"}).reset_index()
for i, p in params.iterrows():
    f.edge(str(p["index"]), '付加価値', label = "{:.3f}".format(p["parameter"]))
f.view()  

/Users/ayuhi2/opt/anaconda3/lib/python3.9/subprocess.py:1052: ResourceWarning: subprocess 78292 is still running
  _warn("subprocess %s is still running" % self.pid,


'path.gv.pdf'

In [18]:
import graphviz
ago = 0
df_fd = getdf()
if ago == -1:
    df_fd = df_fd[df_fd.year <= 2019 + ago]
else:
    df_fd = df_fd[df_fd.year >= 2004 + ago]
df_fd = df_fd.set_index(["id", "year"])
str_ago = "" if ago == 0 else "0" if ago == -1 else ago
year_formula = getYearFormula(ago)
formula_fd = (
            f"value ~ woscinvest{str_ago} + software{str_ago} + income{str_ago} +"
            + year_formula
        )
result_fd = FirstDifferenceOLS.from_formula(formula_fd, data=df_fd).fit(
    cov_type="clustered", cluster_entity=True
)
f = graphviz.Graph('model2', filename='path.gv') 
params = result_fd.params.iloc[:3].rename(index={"income": "従業員給与","software":"ソフトウェア","woscinvest": "設備投資"}).reset_index()
for i, p in params.iterrows():
    f.edge(str(p["index"]), '付加価値', label = "{:.3f}".format(p["parameter"]))
f.view()  

/Users/ayuhi2/opt/anaconda3/lib/python3.9/subprocess.py:1052: ResourceWarning: subprocess 78010 is still running
  _warn("subprocess %s is still running" % self.pid,


'path.gv.pdf'

In [91]:
class estimator:
    def define_formula(self, ago: int):
        df_fd = getdf()
        if ago == -1:
            df_fd = df_fd[df_fd.year <= 2019 + ago]
        else:
            df_fd = df_fd[df_fd.year >= 2004 + ago]
        self.df_fd = df_fd.set_index(["id", "year"])
        # 年のダミー変数を取得
        year_formula = getYearFormula(ago)
        # scale_formula = getScaleDummyformula()
        self.ago = ago
        self.str_ago = "" if ago == 0 else "0" if ago == -1 else ago
        # モデル
        ## 一般
        self.formula_fd = (
            f"value ~ woscinvest{self.str_ago} + software{self.str_ago} + income{self.str_ago} +"
            + year_formula
        )
        self.formula_fe = f"value ~ woscinvest{self.str_ago} + software{self.str_ago} + income{self.str_ago} + EntityEffects + TimeEffects"
        ## 割合
        self.formula_fd_rate = (
            f"value_rate ~ invest_rate{self.str_ago} + soft_rate{self.str_ago} + income_rate{self.str_ago} + "
            + year_formula
        )
        self.formula_fe_rate = f"value_rate ~ invest_rate{self.str_ago} + soft_rate{self.str_ago} + income_rate{self.str_ago}  + EntityEffects + TimeEffects"
        ## 一部差分
        self.formula = f"value_diff ~ software_diff{self.str_ago} + capital + income_diff{self.str_ago} + woscinvest_diff{self.str_ago} + TimeEffects"

    def getParams(self, result, category=None):
        summary = pd.read_html(result.summary.as_html(), header=0, index_col=0)[0]
        summary.replace(r"\(|\)", "", regex=True, inplace=True)
        pvalues = result.pvalues
        # 変数の数を取得
        # 係数部分を抜き出し
        params = result.params
        # 小数点処理と有意マークをつける
        params = params.astype(float).applymap(
            lambda x: "{:.2f}".format(round(x, 3 - math.floor(math.log10(abs(x))) - 1))
            if math.floor(math.log10(abs(x))) > 0
            else "{:.3f}".format(round(x, 3 - math.floor(math.log10(abs(x))) - 1)),
            na_action="ignore",
        ).astype(str) + pvalues.applymap(
            lambda x: " ***"
            if x < 0.01
            else " **"
            if x < 0.05
            else " *"
            if x < 0.1
            else ""
        )
        # 標準偏差
        std = result.std_errors
        std = std.astype(float).applymap(
            lambda x: "{:.2f}".format(x)
            if x == 0
            else "{:.2f}".format(round(x, 3 - math.floor(math.log10(abs(x))) - 1)),
            na_action="ignore",
        )
        # 係数と変数を結合
        if category is None:
            params = pd.concat([params, std], axis=1, keys=["Coef.", "SE"]).swaplevel(
                0, 1, axis=1
            )
        else:
            params = params.add_suffix("_Coef.")
        #  indexソート
        self.r_order = [
            f"software_diff{self.str_ago}",
            "capital",
            f"woscinvest_diff{self.str_ago}",
            f"income_diff{self.str_ago}",
            f"software{self.str_ago}",
            f"woscinvest{self.str_ago}",
            f"income{self.str_ago}",
            f"soft_rate{self.str_ago}",
            f"invest_rate{self.str_ago}",
            f"income_rate{self.str_ago}",
        ]
        params.sort_index(key=lambda x: self.sorter(x, self.r_order), inplace=True)
        # マルチインデックスに対応
        if category is None:
            params.columns = ["_".join(col) for col in params.columns]
        return params

    def getStatics(self, result):
        summary = pd.read_html(result.summary.as_html(), header=0, index_col=0)[0]
        # 　検定量を抜き出す
        static = summary.iloc[:10].drop(summary.index[[0, 1, 3, 6, 7, 8]])

        static.columns = static.columns + "_Coef."
        # 小数点処理
        static.loc[
            (static.index != "No. Observations") & (static.index != "P-value (F-stat)")
        ] = (
            static.loc[
                (static.index != "No. Observations")
                & (static.index != "P-value (F-stat)")
            ]
            .astype(float)
            .applymap(
                lambda x: "{:.3f}".format(x)
                if x == 0
                else "{:.3f}".format(round(x, 3 - math.floor(math.log10(abs(x))) - 1))
                if math.floor(math.log10(abs(x))) == 0
                else round(x, 3 - math.floor(math.log10(abs(x))) - 1),
                na_action="ignore",
            )
        )
        static.loc["No. Observations"] = static.loc["No. Observations"].astype(int)
        static.loc["P-value (F-stat)"] = (
            static.loc["P-value (F-stat)"]
            .astype(float)
            .map(
                lambda x: round(x, 3 - math.floor(math.log10(abs(x))) - 1)
                if x != 0
                else "{:.3f}".format(x),
                na_action="ignore",
            )
        )
        return static

    def getPvalues(self, result):
        pvalues = result.pvalues
        pvalues.applymap(lambda x: "{:.3f}".format(x)).sort_index(
            key=lambda x: self.sorter(x, self.r_order)
        )
        pvalues.index += "_pvalue"
        pvalues.columns = pvalues.columns + "_Coef."
        return pvalues

    def getRenameIndex(self):
        t = (
            ""
            if self.ago == 0
            else f"(t+{-self.ago})"
            if self.ago < 0
            else f"(t{-self.ago})"
        )
        rename_index = {
            f"software{self.str_ago}": f"Software_diff{t}",
            f"wosinfasset{self.str_ago}": f"Intangible_assets_WS.{t}",
            "capital": "Capital",
            f"retain_rate{self.str_ago}": f"Internal_Reserves_Rate{t}",
            f"soft_rate{self.str_ago}": f"Software_Rate{t}",
            f"income_diff{self.str_ago}": f"Salary_diff{t}",
            f"income{self.str_ago}": f"Salary_diff{t}",
            f"woscinvest{self.str_ago}": f"Capital_Invest_WS.{t}",
            f"software_diff{self.str_ago}": f"Software_diff{t}",
            f"woscinvest_diff{self.str_ago}": f"Capital_Invest_WS._diff{t}",
            f"income_rate{self.str_ago}": f"Salary_Rate{t}",
            f"invest_rate{self.str_ago}": f"Capital_Invest_WS._Rate{t}",
        }
        return rename_index

    def getOutput(self, results, category=""):  # type: ignore
        params_list = []
        statics_list = []
        pvalues_list = []
        output = self.df_fd.iloc[0:0]
        if isinstance(results, dict):
            for key in results.keys():
                self.ago = key
                self.str_ago = "" if key == 0 else "0" if key == -1 else key
                params = self.getParams(results[key], category)
                statics = self.getStatics(results[key])
                pvalues = self.getPvalues(results[key])
                # indexのリネーム
                rename_index = self.getRenameIndex()
                params.rename(
                    index=rename_index,
                    inplace=True,
                )
                statics.rename(
                    index=rename_index,
                    inplace=True,
                )
                pvalues.rename(
                    index=rename_index,
                    inplace=True,
                )
                params_list.append(params)
                statics_list.append(statics)
                pvalues_list.append(pvalues)
            df_params = pd.concat(params_list, keys=["t", "t-1", "t-2"], axis=1)
            df_statics = pd.concat(statics_list, keys=["t", "t-1", "t-2"], axis=1)
            df_pvalues = pd.concat(pvalues_list, keys=["t", "t-1", "t-2"], axis=1)
            df_params.columns = ["_".join(col) for col in df_params.columns]
            df_statics.columns = ["_".join(col) for col in df_statics.columns]
            df_pvalues.columns = ["_".join(col) for col in df_pvalues.columns]
            df_params = df_params[~df_params.index.str.startswith("year")]
            df_pvalues = df_pvalues[~df_pvalues.index.str.startswith("year")]
            output = pd.concat(
                [
                    df_params,
                    df_statics,
                    pd.DataFrame(
                        index=[
                            "*** p < 0.01, ** p < 0.05, * p < 0.1, where SD is Rubast SD"
                        ],
                        columns=df_params.columns,
                    ),
                    df_pvalues,
                ]
            )
        else:
            params = self.getParams(results)
            statics = self.getStatics(results)
            pvalues = self.getPvalues(results)
            output = pd.concat(
                [
                    params,
                    statics,
                    pd.DataFrame(
                        index=[
                            "*** p < 0.01, ** p < 0.05, * p < 0.1, where SD is Rubast SD"
                        ],
                        columns=params.columns,
                    ),
                    pvalues,
                ]
            )
            # indexのリネーム
            rename_index = self.getRenameIndex()
            output.rename(
                index=rename_index,
                inplace=True,
            )
        # yearの係数を除外
        output = output[~output.index.str.startswith("year")]
        # マルチインデックス化
        if category != "":
            category = (
                "under 10 million yen"
                if category == 5
                else "10 million yen or more but less than 50 million yen"
                if category == 25
                else "50 million yen or more but less than 100 million yen"
                if category == 75
                else "100 million yen or more but less than 1 billion yen"
                if category == 500
                else "1 billion yen or more"
                if category == 1000
                else category
            )
            output = output.add_prefix(f"{category}_")
        new_list = np.array([col.split("_") for col in output.columns]).T
        col = pd.MultiIndex.from_arrays(new_list)
        output.set_axis(col, axis=1, inplace=True)
        # output = output.swaplevel(0, 1, axis=1)
        l_order1 = ["PartDiff", "General", "Rate"]
        l_order2 = ["t", "t-1", "t-2"]
        output.sort_index(axis=1, key=lambda x: self.sorter(x, l_order1), inplace=True)
        if "PartDiff" in output.columns:
            output.iloc[4, 0:2] = output.iloc[0, 0:2]
            output.iloc[5, 0:2] = output.iloc[2, 0:2]
            output.iloc[6, 0:2] = output.iloc[3, 0:2]
            output.reset_index(inplace=True)
            output.drop([0,2,3], inplace=True)
            output.set_index('index', inplace=True)
        output.replace("nan", "", inplace=True)
        return output

    ######## sort!!!!!
    def sorter(self, x, order):
        if any(x.isin(order)):
            return x.map({s: order.index(s) if s in order else float("inf") for s in x})
        return x

    def fit(self, scale=None, industry=None):
        df_fd = self.df_fd
        if scale:
            df_fd = self.df_fd[self.df_fd["scale"] == scale]
        if industry:
            df_fd = self.df_fd[self.df_fd["industry_name"] == industry]
        # モデルフィト
        ## 一般
        result_fd = FirstDifferenceOLS.from_formula(self.formula_fd, data=df_fd).fit(
            cov_type="clustered", cluster_entity=True
        )
        result_fe = PanelOLS.from_formula(self.formula_fe, data=df_fd).fit(
            cov_type="clustered", cluster_entity=True
        )
        ## 割合
        result_fd_rate = FirstDifferenceOLS.from_formula(
            self.formula_fd_rate, data=df_fd
        ).fit(cov_type="clustered", cluster_entity=True)
        result_fe_rate = PanelOLS.from_formula(self.formula_fe_rate, data=df_fd).fit(
            cov_type="clustered", cluster_entity=True
        )
        result = compare(
            {
                "General_FE": result_fd,
                "General_FD": result_fe,
                "Rate_FE": result_fd_rate,
                "Rate_FD": result_fe_rate,
            }
        )
        ## 一部差分
        if scale is None and industry is None:
            result_OSL = PanelOLS.from_formula(
                self.formula, df_fd, other_effects=df_fd["industry"]
            ).fit(cov_type="clustered", cluster_entity=True)
            result = compare(
                {
                    "PartDiff_OLS": result_OSL,
                    "General_FD": result_fd,
                    "General_FE": result_fe,
                    "Rate_FD": result_fd_rate,
                    "Rate_FE": result_fe_rate,
                }
            )
        return result

    def run(self):
        # 保存ファイル名
        self.file_name = f"f.xlsx"
        print("-------------------")
        print("全体")
        for i in np.arange(-1, 3):
            self.define_formula(i)
            result = self.fit()
            output = self.getOutput(result)
            # 保存
            if i == -1:
                output.to_excel(
                    self.file_name,
                    sheet_name=f"all{self.ago}",
                    encoding="shift-jis",
                    float_format="%.3f",
                )
            else:
                with pd.ExcelWriter(self.file_name, mode="a") as writer:
                    output.to_excel(
                        writer,
                        encoding="shift-jis",
                        sheet_name=f"all{self.ago}",
                        float_format="%.3f",
                    )
        # 規模別
        for scale in np.sort(df["scale"].unique()):
            print(scale)
            scale_results = {}
            for i in np.arange(0, 3):
                self.define_formula(i)
                result = self.fit(scale=scale)
                scale_results[i] = result
            output = self.getOutput(scale_results, scale)
            with pd.ExcelWriter(self.file_name, mode="a") as writer:
                output.to_excel(
                    writer,
                    encoding="shift-jis",
                    sheet_name=f"scale{scale}",
                    float_format="%.3f",
                )
        # 業種別
        for industry in df["industry_name"].unique():
            print(industry)
            industry_results = {}
            for i in np.arange(0, 3):
                self.define_formula(i)
                result = self.fit(industry=industry)
                industry_results[i] = result
            output = self.getOutput(industry_results, industry)
            with pd.ExcelWriter(self.file_name, mode="a") as writer:
                output.to_excel(
                    writer,
                    encoding="shift-jis",
                    sheet_name=f"industry{industry}",
                    float_format="%.3f",
                )


est = estimator()
est.run()

-------------------
全体
                               PartDiff      
                                    OLS      
                                  Coef.    SE
Software_diff(t+1)               -0.220  0.20
Capital                       0.071 ***  0.02
Capital_Invest_WS._diff(t+1)      0.013  0.01
Salary_diff(t+1)               -0.088 *  0.05
Software_diff(t+1)               -0.220  0.20
Capital_Invest_WS.(t+1)           0.013  0.01
Salary_diff(t+1)               -0.088 *  0.05
                          PartDiff      
                               OLS      
                             Coef.    SE
Software_diff            1.380 ***  0.50
Capital                   0.017 **  0.01
Capital_Invest_WS._diff  0.055 ***  0.01
Salary_diff              1.570 ***  0.05
Software_diff            1.380 ***  0.50
Capital_Invest_WS.       0.055 ***  0.01
Salary_diff              1.570 ***  0.05
                               PartDiff      
                                    OLS      
               

In [309]:
class estimator:
    def __init__(self, ago: int):
        df_fd = getdf()
        if ago == -1:
            df_fd = df_fd[df_fd.year <= 2019 + ago]
        else:
            df_fd = df_fd[df_fd.year >= 2004 + ago]
        self.df_fd = df_fd.set_index(["id", "year"])
        # 年のダミー変数を取得
        year_formula = getYearFormula(ago)
        # scale_formula = getScaleDummyformula()
        self.ago = ago
        self.str_ago = "" if ago == 0 else '0' if ago == -1 else ago
        # モデル
        ## 一般
        self.formula_fd = (
            f"value ~ woscinvest{self.str_ago} + software{self.str_ago} + income_diff{self.str_ago} +"
            + year_formula
        )
        self.formula_fe = f"value ~ woscinvest{self.str_ago} + software{self.str_ago} + income_diff{self.str_ago} + EntityEffects + TimeEffects"
        ## 割合
        self.formula_fd_rate = (
            f"value_rate ~ invest_rate{self.str_ago} + soft_rate{self.str_ago} + income_rate{self.str_ago} + "
            + year_formula
        )
        self.formula_fe_rate = f"value_rate ~ invest_rate{self.str_ago} + soft_rate{self.str_ago} + income_rate{self.str_ago}  + EntityEffects + TimeEffects"
        ## 一部差分
        self.formula = f"value_diff ~ software_diff{self.str_ago} + capital + income + woscinvest_diff{self.str_ago} + TimeEffects"
        # 保存ファイル名
        self.file_name = f"f{self.str_ago}.xlsx"

    def getOutput(self, summary, pvalues, factor_num: int):  # type: ignore
        # ()を削除
        summary.replace(r"\(|\)", "", regex=True, inplace=True)
        ago = 1 if self.ago == -1 else self.ago
        result_num = 11 + ((15 - ago) * 2) + factor_num * 2
        # 変数の数を取得
        # 係数部分を抜き出し
        params = summary.iloc[11:result_num:2]
        # 小数点処理と有意マークをつける
        params = params.astype(float).applymap(
            lambda x: round(x, 3 - math.floor(math.log10(abs(x))) - 1)
            if math.floor(math.log10(abs(x))) != 0
            else "{:.2f}".format(round(x, 3 - math.floor(math.log10(abs(x))) - 1)),
            na_action="ignore",
        ).astype(str) + pvalues.applymap(
            lambda x: " ***"
            if x < 0.01
            else " **"
            if x < 0.05
            else " *"
            if x < 0.1
            else ""
        )
        # 標準偏差
        std = (
            summary.iloc[12:result_num:2].reset_index(drop=True).set_axis(params.index)
        )
        std = std.astype(float).applymap(
            lambda x: "{:.2f}".format(x)
            if x == 0
            else "{:.2f}".format(round(x, 3 - math.floor(math.log10(abs(x))) - 1))
            if math.floor(math.log10(abs(x))) == 0
            else "{:.1f}".format(round(x, 3 - math.floor(math.log10(abs(x))) - 1)),
            na_action="ignore",
        )
        # 係数と変数を結合
        params = pd.concat([params, std], axis=1, keys=["係数", "標準誤差"]).swaplevel(
            0, 1, axis=1
        )
        #  indexソート
        r_order = [
            f"software_diff{self.str_ago}",
            "capital",
            f"woscinvest_diff{self.str_ago}",
            "income",
            f"software{self.str_ago}",
            f"woscinvest{self.str_ago}",
            f"income_diff{self.str_ago}",
        ]
        params.sort_index(key=lambda x: self.sorter(x, r_order), inplace=True)
        # 　検定量を抜き出す
        static = summary.iloc[:10].drop(summary.index[[0, 1, 3, 6, 7, 8]])
        # 検定量とp値を結合
        static = pd.concat(
            [
                static,
                pd.DataFrame(index=[""], columns=summary.columns),
                pvalues.applymap(lambda x: "{:.3f}".format(x)).sort_index(
                    key=lambda x: self.sorter(x, r_order)
                ),
            ]
        )
        static.columns = static.columns + "_係数"
        # 小数点処理
        static.loc[
            (static.index != "No. Observations") & (static.index != "P-value (F-stat)")
        ] = (
            static.loc[
                (static.index != "No. Observations")
                & (static.index != "P-value (F-stat)")
            ]
            .astype(float)
            .applymap(
                lambda x: "{:.3f}".format(x)
                if x == 0
                else "{:.3f}".format(round(x, 3 - math.floor(math.log10(abs(x))) - 1))
                if math.floor(math.log10(abs(x))) == 0
                else round(x, 3 - math.floor(math.log10(abs(x))) - 1),
                na_action="ignore",
            )
        )
        static.loc["No. Observations"] = static.loc["No. Observations"].astype(int)
        static.loc["P-value (F-stat)"] = (
            static.loc["P-value (F-stat)"]
            .astype(float)
            .map(
                lambda x: round(x, 3 - math.floor(math.log10(abs(x))) - 1)
                if x != 0
                else "{:.3f}".format(x),
                na_action="ignore",
            )
        )
        # マルチインデックスに対応
        params.columns = ["_".join(col) for col in params.columns]
        # yearの係数を除外
        params = params[~params.index.str.startswith("year")]
        static = static[~static.index.str.startswith("year")]
        output = pd.concat([params, static])
        # マルチインデックス化
        new_list = np.array([col.split("_") for col in output.columns]).T
        col = pd.MultiIndex.from_arrays(new_list)
        output.set_axis(col, axis=1, inplace=True)
        output = output.swaplevel(0, 1, axis=1)
        l_order = ["一部差分", "一般", "割合"]
        output.sort_index(axis=1, key=lambda x: self.sorter(x, l_order), inplace=True)
        if "一部差分" in output.columns:
            output.iloc[4, 0:2] = output.iloc[0, 0:2]
            output.drop(output.index[0], inplace=True)

        # indexのリネーム
        t = '' if self.ago == 0 else f'(t+{-self.ago})' if self.ago < 0 else f'(t{-self.ago})'
        output.rename(
            index={
                "No. Observations": "サンプルサイズ",
                "R-squared": "R2乗決定係数",
                "R-Squared (Within)": "R2乗決定係数(Within)",
                "P-value (F-stat)": "P値 (F統計量)",
                f"software{self.str_ago}": f"ソフトウェアの変化{t}",
                f"wosinfasset{self.str_ago}": f"ソフトウェア除く\n無形固定資産{t}",
                "capital": "資本金",
                f"retain_rate{self.str_ago}": f"内部留保率{t}",
                f"soft_rate{self.str_ago}": f"売上に占めるソフトウェアの割合{t}",
                f"ltfasset{self.str_ago}": f"有形固定資産(log){t}",
                f"retain{self.str_ago}": f"社内留保{t}",
                f"tfasset{self.str_ago}": f"有形固定資産{t}",
                f"income_diff{self.str_ago}": f"従業員給与の変化{t}",
                "income": "従業員給与",
                f"woscinvest{self.str_ago}": f"ソフトウェアを除く設備投資{t}",
                f"software_diff{self.str_ago}": f"ソフトウェアの変化{t}",
                f"woscinvest_diff{self.str_ago}": f"ソフトウェアを除く設備投資の変化{t}",
                f"income_rate{self.str_ago}": f"売上に占める従業員給与の割合{t}",
                f"invest_rate{self.str_ago}": f"売上に占める設備投資の割合{t}",
            },
            inplace=True,
        )
        return output

    ######## sort!!!!!
    def sorter(self, x, order):
        if any(x.isin(order)):
            return x.map({s: order.index(s) if s in order else float("inf") for s in x})
        return x

    # type: ignore
    def fit(self, scale=None, industry=None):
        df_fd = self.df_fd
        if scale:
            df_fd = self.df_fd[self.df_fd["scale"] == scale]
        if industry:
            df_fd = self.df_fd[self.df_fd["industry_name"] == industry]
        factor_list = (
            set(self.formula_fe.split(" + "))
            | set(self.formula_fe_rate.split(" + "))
            | set(self.formula.split(" + "))
        )
        if scale or industry:
            factor_list = set(self.formula_fe.split(" + ")) | set(
                self.formula_fe_rate.split(" + ")
            )
        factor_num = len(factor_list) - 2
        # モデルフィト
        ## 一般
        result_fd = FirstDifferenceOLS.from_formula(self.formula_fd, data=df_fd).fit(
            cov_type="clustered", cluster_entity=True
        )
        result_fe = PanelOLS.from_formula(self.formula_fe, data=df_fd).fit(
            cov_type="clustered", cluster_entity=True
        )
        ## 割合
        result_fd_rate = FirstDifferenceOLS.from_formula(
            self.formula_fd_rate, data=df_fd
        ).fit(cov_type="clustered", cluster_entity=True)
        result_fe_rate = PanelOLS.from_formula(self.formula_fe_rate, data=df_fd).fit(
            cov_type="clustered", cluster_entity=True
        )
        result = compare(
            {
                "一階差分推定_一般": result_fd,
                "固定効果推定_一般": result_fe,
                "一階差分推定_割合": result_fd_rate,
                "固定効果推定_割合": result_fe_rate,
            }
        )
        ## 一部差分
        if scale is None and industry is None:
            result_OSL = PanelOLS.from_formula(
                self.formula, df_fd, other_effects=df_fd["industry"]
            ).fit(cov_type="clustered", cluster_entity=True)
            result = compare(
                {
                    "OLS推定_一部差分": result_OSL,
                    "一階差分推定_一般": result_fd,
                    "固定効果推定_一般": result_fe,
                    "一階差分推定_割合": result_fd_rate,
                    "固定効果推定_割合": result_fe_rate,
                }
            )
        summary = pd.read_html(result.summary.as_html(), header=0, index_col=0)[0]
        # table作成
        output = self.getOutput(summary, result.pvalues, factor_num)
        output.replace(np.nan, None, inplace=True)
        output.replace("nan", "", inplace=True)
        output.fillna("", inplace=True)
        output.loc["** p < 0.01, * p < 0.05, ここでの標準誤差は頑健標準誤差"] = np.nan
        print(output)
        return output

    def run(self):
        industry_list = []
        scale_list = []
        print("-------------------")
        print("全体")
        output = self.fit()
        # 保存
        output.to_excel(self.file_name, encoding="shift-jis", float_format="%.3f")
        # 規模別
        for scale in np.sort(df["scale"].unique()):
            print(scale)
            output = self.fit(scale=scale)
            scale_list.append(output)
        df_csv = pd.concat(
            scale_list,
            axis=1,
            keys=["1千万円未満", "1千万円以上5千万未満", "5千万円以上1億未満", "1億円以上10億未満", "10億以上"],
        )
        df_csv.loc["** p < 0.01, * p < 0.05, ここでの標準誤差は頑健標準誤差"] = np.nan
        # print(df_csv)
        with pd.ExcelWriter(self.file_name, mode="a") as writer:
            df_csv.to_excel(
                writer,
                encoding="shift-jis",
                sheet_name="new_sheet1",
                float_format="%.3f",
            )
        # 業種別
        for industry in df["industry_name"].unique():
            print(industry)
            output = self.fit(industry=industry)
            industry_list.append(output)
        df_csv = pd.concat(
            industry_list,
            axis=1,
            keys=df["industry_name"].unique().map(lambda x: "・".join(x.split("_"))),
        )
        df_csv.loc["** p < 0.01, * p < 0.05, ここでの標準偏差は頑健標準誤差"] = np.nan
        # print(df_csv)
        with pd.ExcelWriter(self.file_name, mode="a") as writer:
            df_csv.to_excel(
                writer,
                encoding="shift-jis",
                sheet_name="new_sheet2",
                float_format="%.3f",
            )


# for i in range(3):
# 	f_fit(i, rate=True)
for i in np.arange(-1, 2):
    est = estimator(i)
    est.run()

-------------------
全体
                                               一部差分                一般         \
                                              OLS推定            一階差分推定          
                                                 係数   標準誤差         係数   標準誤差   
資本金                                         -0.0191   -0.6                     
ソフトウェアを除く設備投資の変化(t+1)                        0.0094   1.15                     
従業員給与                                     0.207 ***   4.22                     
ソフトウェアの変化(t+1)                               -0.215  -1.17     -0.257  -1.16   
ソフトウェアを除く設備投資(t+1)                                             0.0116   1.36   
従業員給与の変化(t+1)                                               -0.0916 *  -1.80   
売上に占める従業員給与の割合(t+1)                                                            
売上に占める設備投資の割合(t+1)                                                             
売上に占めるソフトウェアの割合(t+1)                                                           
サンプルサイズ          